## Loading the data

In [1]:
!pip install datasets
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 61.7 MB/s 
     |████████████████████████████████| 140 kB 53.8 MB/s 
     |████████████████████████████████| 212 kB 71.2 MB/s 
     |████████████████████████████████| 86 kB 7.8 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 596 kB 64.0 MB/s 
     |████████████████████████████████| 127 kB 57.1 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 65.7 MB/s 
     |████████████████████████████████| 271 kB 56.1 MB/s 
     |████████████████████████████████| 112 kB 67.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing ins

In [1]:
from google.colab import files
 
#Upload OpArticles/OpArticles_ADU.xlsx
uploaded = files.upload()

Saving OpArticles_ADUs.xlsx to OpArticles_ADUs.xlsx


In [6]:
import pandas as pd
import io 
from sklearn import preprocessing

dataset_ADU = pd.read_excel(io.BytesIO(uploaded['OpArticles_ADU.xlsx']))

label_encoder = preprocessing.LabelEncoder()

dataset_ADU['label']= label_encoder.fit_transform(dataset_ADU['label'])

dataset_ADU.pop("article_id")
dataset_ADU.pop("annotator")
dataset_ADU.pop("node")
dataset_ADU.pop("ranges")

dataset_ADU.head()

,tokens,label
0,"""porque é mais prático e, o que é pior, para t...",0
1,"""revolução"" é um termo frequente na literatura...",0
2,"(In)felizmente, essas relações acabam, mais ce...",0
3,(em 2017 foram quatro vezes maiores,0
4,(pelo que julgo poder deduzir) não está obriga...,0


In [7]:
from datasets import Dataset

dataset_hf = Dataset.from_pandas(dataset_ADU)

dataset_hf

Dataset({
    features: ['tokens', 'label'],
    num_rows: 12008
})

In [8]:
from datasets import DatasetDict

train_test = dataset_hf.train_test_split(test_size=0.1)

valid_test = train_test['test'].train_test_split(test_size=0.5)

train_valid_test_dataset = DatasetDict({
  'train': train_test['train'],
  'validation': valid_test['train'],
  'test': valid_test['test']
})



## Finetuning a pretained model

In [19]:
model_name = "neuralmind/bert-large-portuguese-cased"

In [20]:
from transformers import  AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(model_name)

https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp1as151d5


Downloading:   0%|          | 0.00/155 [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/3a44fa9a74e90f509368a7f2789df38e1fedd153a52c62ef5cc5f4b0f5c99c2a.d61b68f744aef2741575c270d4ba0228cd35693bfa15d8babfb5c1079062d5d7
creating metadata file for /root/.cache/huggingface/transformers/3a44fa9a74e90f509368a7f2789df38e1fedd153a52c62ef5cc5f4b0f5c99c2a.d61b68f744aef2741575c270d4ba0228cd35693bfa15d8babfb5c1079062d5d7
https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp7jwc_q6w


Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
creating metadata file for /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
loading configuration file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-large-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  

...

storing https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/9cfcd25de0a333b1b5f4a3db227e93a806cfb041d93a49221eeaee6773eaa41c.af25fb1e29ad0175300146695fd80069be69b211c52fa5486fa8aae2754cc814
creating metadata file for /root/.cache/huggingface/transformers/9cfcd25de0a333b1b5f4a3db227e93a806cfb041d93a49221eeaee6773eaa41c.af25fb1e29ad0175300146695fd80069be69b211c52fa5486fa8aae2754cc814
https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/added_tokens.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp7p493tgo


Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/added_tokens.json in cache at /root/.cache/huggingface/transformers/6a3aa038873b8f0d0ab3a4de0a658f063b89e3afd815920a5f393c0e4ae84259.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
creating metadata file for /root/.cache/huggingface/transformers/6a3aa038873b8f0d0ab3a4de0a658f063b89e3afd815920a5f393c0e4ae84259.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp3ch501cg


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/d5b721c156180bbbcc4a1017e8c72a18f8f96cdc178acec5ddcd45905712b4cf.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/d5b721c156180bbbcc4a1017e8c72a18f8f96cdc178acec5ddcd45905712b4cf.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/9cfcd25de0a333b1b5f4a3db227e93a806cfb041d93a49221eeaee6773eaa41c.af25fb1e29ad0175300146695fd80069be69b211c52fa5486fa8aae2754cc814
loading file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/added_tokens.json from cache

In [21]:
def preprocess_function(sample):
    return tokenizer(sample["tokens"], truncation=True, padding=True)

In [22]:
tokenized_dataset = train_valid_test_dataset.map(preprocess_function, batched=True)

tokenized_dataset

  0%|          | 0/11 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10807
    })
    validation: Dataset({
        features: ['tokens', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 600
    })
    test: Dataset({
        features: ['tokens', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 601
    })
})

In [23]:
from transformers import AutoModel

model = AutoModel.from_pretrained(model_name)

loading configuration file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-large-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_t

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/016fb7702039667c9fb9dd2ceffaf04027b13e525a6248cda2a4a87dbb8687af.881d7200bce807f871637ac9d552c541b2d4b00146a0bf1ab0360f3640031273
creating metadata file for /root/.cache/huggingface/transformers/016fb7702039667c9fb9dd2ceffaf04027b13e525a6248cda2a4a87dbb8687af.881d7200bce807f871637ac9d552c541b2d4b00146a0bf1ab0360f3640031273
loading weights file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/016fb7702039667c9fb9dd2ceffaf04027b13e525a6248cda2a4a87dbb8687af.881d7200bce807f871637ac9d552c541b2d4b00146a0bf1ab0360f3640031273
Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bi

In [24]:
from transformers import AutoModelForSequenceClassification



model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, ignore_mismatched_sizes=True)


loading configuration file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-large-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads

## Fine Tuning

In [25]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from datasets import load_metric
import numpy as np

metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    load_best_model_at_end=True,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [26]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10807
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2028


Epoch,Training Loss,Validation Loss,Accuracy
1,1.029000,0.897859,0.598333
2,0.783400,0.932736,0.625000
3,0.443400,1.118424,0.620000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 600
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-676
Configuration saved in ./results/checkpoint-676/config.json
Model weights saved in ./results/checkpoint-676/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-676/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-676/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 600
  Batch size = 16
Sa

TrainOutput(global_step=2028, training_loss=0.7176320468180278, metrics={'train_runtime': 3078.7951, 'train_samples_per_second': 10.53, 'train_steps_per_second': 0.659, 'total_flos': 9022250033153136.0, 'train_loss': 0.7176320468180278, 'epoch': 3.0})

In [27]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 600
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5983333333333334,
 'eval_loss': 0.8978592753410339,
 'eval_runtime': 9.4022,
 'eval_samples_per_second': 63.815,
 'eval_steps_per_second': 4.042}

In [28]:
trainer.predict(test_dataset=tokenized_dataset["test"])

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 601
  Batch size = 16


PredictionOutput(predictions=array([[ 0.66116965, -2.8653998 ,  2.3203337 , -1.281385  ,  0.43902072],
       [ 1.8257824 , -2.849616  ,  1.5710397 ,  1.8235954 , -2.3044236 ],
       [-1.2915916 ,  1.7722172 ,  2.137578  , -0.81834584, -1.1060268 ],
       ...,
       [ 0.3073924 , -2.4624774 ,  1.5834275 ,  1.9968183 , -1.9813766 ],
       [-0.06478488, -2.7524965 ,  1.8926104 , -2.1943932 ,  3.3400972 ],
       [ 1.2651017 , -2.7227242 ,  1.9332601 ,  0.97845054, -1.544646  ]],
      dtype=float32), label_ids=array([0, 0, 2, 3, 4, 2, 2, 2, 2, 2, 2, 4, 2, 0, 0, 2, 2, 4, 2, 1, 1, 0,
       2, 4, 2, 1, 0, 4, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 4, 1,
       2, 2, 2, 4, 2, 2, 2, 4, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 2,
       0, 4, 0, 4, 0, 2, 0, 4, 0, 0, 4, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 3,
       2, 4, 3, 2, 2, 4, 2, 0, 2, 0, 4, 2, 0, 0, 2, 2, 3, 4, 2, 4, 2, 2,
       2, 0, 2, 2, 2, 3, 2, 2, 2, 3, 4, 2, 4, 4, 0, 3, 2, 4, 2, 2, 0, 2,
       4, 2, 2, 2, 0, 2, 4, 0, 4, 2, 2, 0, 2